In [0]:
#This class has been copied from the tutorial in course
import numpy as np
import tensorflow as tf
import pandas as pd

In [0]:
# Create a class that will do the batching for the algorithm
# This code is extremely reusable. You should just change Audiobooks_data everywhere in the code
class Data_Reader():
    # Dataset is a mandatory arugment, while the batch_size is optional
    # If you don't input batch_size, it will automatically take the value: None
    def __init__(self, dataset, batch_size = None):
    
        # The dataset that loads is one of "train", "validation", "test".
        # e.g. if I call this class with x('train',5), it will load 'Audiobooks_data_train.npz' with a batch size of 5.
        npz = np.load('Audiobook_{0}_Data.npz'.format(dataset))
        
        # Two variables that take the values of the inputs and the targets. Inputs are floats, targets are integers
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        # Counts the batch number, given the size you feed it later
        # If the batch size is None, we are either validating or testing, so we want to take the data in a single batch
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    
    # A method which loads the next batch
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
        # You slice the dataset in batches and then the "next" function loads them one after the other
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        # One-hot encode the targets. In this example it's a bit superfluous since we have a 0/1 column 
        # as a target already but we're giving you the code regardless, as it will be useful for any 
        # classification task with more than one target column
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        # The function will return the inputs batch and the one-hot encoded targets
        return inputs_batch, targets_one_hot
    
        
    # A method needed for iterating over the batches, as we will put them in a loop
    # This tells Python that the class we're defining is iterable, i.e. that we can use it like:
    # for input, output in data: 
        # do things
    # An iterator in Python is a class with a method __next__ that defines exactly how to iterate through its objects
    def __iter__(self):
        return self

In [0]:
######outline model
#10 columns in data set so 10 inputs, 2 possible targets... we will use 2 hidden layers initially with units 50 each

input_size = 10
output_size = 2
hidden_layer_size = 50

tf.reset_default_graph()

inputs_placeholder = tf.placeholder(tf.float32,[None,input_size])
targets_placeholder = tf.placeholder(tf.int32,[None,output_size])


#hidden layer 1
weights_1 = tf.get_variable("weigths_1",[input_size,hidden_layer_size]) #if initialized this way, the default initializer is xavier
biases_1 = tf.get_variable("biases_1",[hidden_layer_size])
output_1 = tf.nn.sigmoid(tf.matmul(inputs_placeholder,weights_1) + biases_1)

#hidden layer 2
weights_2 = tf.get_variable("weigths_2",[hidden_layer_size,hidden_layer_size]) #if initialized this way, the default initializer is xavier
biases_2 = tf.get_variable("biases_2",[hidden_layer_size])
output_2 = tf.nn.sigmoid(tf.matmul(output_1,weights_2) + biases_2)

#hidden layer 3
weights_3 = tf.get_variable("weigths_3",[hidden_layer_size,hidden_layer_size]) #if initialized this way, the default initializer is xavier
biases_3 = tf.get_variable("biases_3",[hidden_layer_size])
output_3 = tf.nn.sigmoid(tf.matmul(output_2,weights_3) + biases_3)

#output layer
weights_4 = tf.get_variable("weigths_4",[hidden_layer_size,output_size]) #if initialized this way, the default initializer is xavier
biases_4 = tf.get_variable("biases_4",[output_size])
outputs = tf.matmul(output_3,weights_4) + biases_4

In [0]:
#calculate loss/delta
loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs,labels=targets_placeholder)

mean_loss = tf.reduce_mean(loss)

In [0]:
#optimize
learning_rate = 0.001
optimize = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(mean_loss)

In [0]:
#test accuracy
out_equals_target = tf.equal(tf.argmax(outputs,1),tf.argmax(targets_placeholder,1))
#out_equals_target is a vector containing 1 if accurately predcted else 0. the accuracy is the mean of this vector
accuracy = tf.reduce_mean(tf.cast(out_equals_target,tf.float32))

In [437]:
#create session var
session = tf.InteractiveSession()
#initialize variables
initializer = tf.global_variables_initializer()
session.run(initializer)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
#batching
batch_size = 50
#batch number is handled in Data_Reader class
max_epoch = 100
prev_validation_loss = 9999999999.

#get data
training_data = Data_Reader('Training',batch_size)
validation_data = Data_Reader('Validation',batch_size)

In [439]:
#train model
for epoch in range(max_epoch):
  curr_epoch_loss = 0
  #train
  for input_batch, target_batch in training_data:
    _, batch_loss = session.run([optimize,mean_loss], feed_dict = {inputs_placeholder: input_batch, targets_placeholder:target_batch})
    curr_epoch_loss += batch_loss
  
  curr_epoch_loss /= training_data.batch_count
  
  #validate
  validation_loss = 0.
  validation_accuracy = 0.
  for input_batch, target_batch in validation_data: #this will always have a single iteration but easy way to get input and target batches
    validation_loss, validation_accuracy = session.run([mean_loss,accuracy], feed_dict = {inputs_placeholder: input_batch, targets_placeholder:target_batch})
  
  
  print('Epoch '+str(epoch+1)
         + '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)
         +'. Validation loss: '+'{0:.3f}'.format(validation_loss)
         +'. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
  #early stop
  if validation_loss > prev_validation_loss:
    break
    
  prev_validation_loss = validation_loss
  
print('End of Training')

Epoch 1. Training loss: 0.716. Validation loss: 0.687. Validation accuracy: 48.00%
Epoch 2. Training loss: 0.675. Validation loss: 0.660. Validation accuracy: 58.00%
Epoch 3. Training loss: 0.611. Validation loss: 0.568. Validation accuracy: 66.00%
Epoch 4. Training loss: 0.500. Validation loss: 0.461. Validation accuracy: 72.00%
Epoch 5. Training loss: 0.433. Validation loss: 0.404. Validation accuracy: 78.00%
Epoch 6. Training loss: 0.406. Validation loss: 0.381. Validation accuracy: 76.00%
Epoch 7. Training loss: 0.391. Validation loss: 0.371. Validation accuracy: 80.00%
Epoch 8. Training loss: 0.382. Validation loss: 0.365. Validation accuracy: 78.00%
Epoch 9. Training loss: 0.376. Validation loss: 0.361. Validation accuracy: 82.00%
Epoch 10. Training loss: 0.372. Validation loss: 0.358. Validation accuracy: 82.00%
Epoch 11. Training loss: 0.368. Validation loss: 0.357. Validation accuracy: 82.00%
Epoch 12. Training loss: 0.365. Validation loss: 0.355. Validation accuracy: 76.00%
E

In [440]:
#test the model

test_data = Data_Reader('Test',batch_size)
for input_batch, target_batch in test_data: 
  test_accuracy = session.run([accuracy], feed_dict = {inputs_placeholder: input_batch, targets_placeholder:target_batch})
    
print('{0:3f}'.format(test_accuracy[0]*100) + '%')

86.000001%


In [0]:
#after tuning and playing around with the hyperparams:
#Accuracy = 86%

#Hyperparams:
#batchsize = 50
#hidden layer = 3
#hidden_layer_size = 50
#learning rate = 0.001